# Timing comparisons with lacosmic and astroscrappy

In this notebook, we compare dfcosmic (both CPU and GPU implementations) with [lacosmic](https://github.com/larrybradley/lacosmic) and [astroscrappy](https://github.com/astropy/astroscrappy). 

These tests were run using a System76 Adder WS with a Intel® Core™ i9-14900HX × 32 and a NVIDIA GeForce RTX 4060 Laptop GPU.

The code snippet for generating fake data was taken **directly** from [astroscrappy tests](https://github.com/astropy/astroscrappy/blob/main/astroscrappy/tests/fake_data.py); therefore, we thank them.

The fake data is set to have the approximate size as a standard narrowband observation taken with Mothra (6000 x 4000).

In [1]:
import time
import numpy as np
from lacosmic.core import lacosmic
from astroscrappy import detect_cosmics
from dfcosmic.core import lacosmic as df_lacosmic

In [2]:
# Make a simple Gaussian function for testing purposes
def gaussian(image_shape, x0, y0, brightness, fwhm):
    x = np.arange(image_shape[1])
    y = np.arange(image_shape[0])
    x2d, y2d = np.meshgrid(x, y)

    sig = fwhm / 2.35482

    normfactor = brightness / 2.0 / np.pi * sig ** -2.0
    exponent = -0.5 * sig ** -2.0
    exponent *= (x2d - x0) ** 2.0 + (y2d - y0) ** 2.0

    return normfactor * np.exp(exponent)


def make_fake_data(size=(6000, 4000)):
    """
    Generate fake data that can be used to test the detection and cleaning algorithms

    Returns
    -------
    imdata : numpy float array
        Fake Image data
    crmask : numpy boolean array
        Boolean mask of locations of injected cosmic rays
    """
    # Set a seed so that the tests are repeatable
    np.random.seed(200)

    # Create a simulated image to use in our tests
    imdata = np.zeros(size, dtype=np.float32)

    # Add sky and sky noise
    imdata += 200

    psf_sigma = 3.5

    # Add some fake sources
    for i in range(100):
        x = np.random.uniform(low=0.0, high=1001)
        y = np.random.uniform(low=0.0, high=1001)
        brightness = np.random.uniform(low=1000., high=30000.)
        imdata += gaussian(imdata.shape, x, y, brightness, psf_sigma)

    # Add the poisson noise
    imdata = np.float32(np.random.poisson(imdata))

    # Add readnoise
    imdata += np.random.normal(0.0, 10.0, size=size)

    # Add 100 fake cosmic rays
    cr_x = np.random.randint(low=5, high=995, size=100)
    cr_y = np.random.randint(low=5, high=995, size=100)

    cr_brightnesses = np.random.uniform(low=1000.0, high=30000.0, size=100)

    imdata[cr_y, cr_x] += cr_brightnesses
    imdata = imdata.astype('f4')

    # Make a mask where the detected cosmic rays should be
    crmask = np.zeros(size, dtype=bool)
    crmask[cr_y, cr_x] = True
    return imdata, crmask


In [3]:
# Make fake data
imdata, expected_crmask = make_fake_data()

Run the code using our four options:

1. dfcosmic on the CPU
2. dfcosmic on the GPU
3. astroscrappy
4. lacosmic

In [4]:
start_dfcosmic_cpu = time.perf_counter()
clean, crmask = df_lacosmic(
            image=imdata,
            objlim=2,
            sigfrac=1,
            sigclip=6,
            gain=1,
            readnoise=10,
            niter=1,
            device='cpu'
        )

elapsed_dfcosmic_cpu = time.perf_counter() - start_dfcosmic_cpu


/home/carterrhea/Documents/dfcosmic/src/dfcosmic/utils.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  block_size_int = torch.tensor(block_size, dtype=torch.int)


In [5]:
start_dfcosmic_gpu = time.perf_counter()
clean, crmask = df_lacosmic(
            image=imdata,
            objlim=2,
            sigfrac=1,
            sigclip=6,
            gain=1,
            readnoise=10,
            niter=1,
            device='cuda'
        )

elapsed_dfcosmic_gpu = time.perf_counter() - start_dfcosmic_gpu


In [6]:
start_astroscrappy = time.perf_counter()
mask, _clean = detect_cosmics(imdata, readnoise=10., gain=1.0,
                                  sigclip=6, sigfrac=1.0)

elapsed_astroscrappy = time.perf_counter() - start_astroscrappy


In [7]:
start_lacosmic = time.perf_counter()
clean, crmask = lacosmic(
            data=imdata,
            contrast=2,
            neighbor_threshold=1,
            cr_threshold=6,
            effective_gain=1,
            readnoise=10,
            maxiter=1,
        )

elapsed_lacosmic = time.perf_counter() - start_lacosmic


INFO: Iteration 1: Found 162 cosmic-ray pixels, Total: 162 [lacosmic.core]


Print out runtimes

In [8]:
print(f"dfcosmic CPU runtime: {elapsed_dfcosmic_cpu:.4f} seconds")
print(f"dfcosmic GPU runtime: {elapsed_dfcosmic_gpu:.4f} seconds")
print(f"astroscrappy runtime: {elapsed_astroscrappy:.4f} seconds")
print(f"lacosmic runtime: {elapsed_lacosmic:.4f} seconds")

dfcosmic CPU runtime: 5.6413 seconds
dfcosmic GPU runtime: 2.3594 seconds
astroscrappy runtime: 2.8420 seconds
lacosmic runtime: 24.8594 seconds
